In [3]:
import pandas as pd
import numpy as np


In [16]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower, \
    macd.`macd_histogram` as macd, ttma.macd_histogram as wave_a, ttmb.macd_histogram as wave_b, ttmc.macd_histogram as wave_c \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 180) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.macd` as macd on macd.symbol = stock.symbol and macd.effective_date = stock.effective_date \
        and macd.short_period = 12 and macd.long_period=26 and macd.signal_period=9 \
    LEFT JOIN \
    `idc.macd` as ttma on ttma.symbol = stock.symbol and ttma.effective_date = stock.effective_date \
        and ttma.short_period = 8 and ttma.long_period=34 and ttma.signal_period=34 \
    LEFT JOIN \
    `idc.macd` as ttmb on ttmb.symbol = stock.symbol and ttmb.effective_date = stock.effective_date \
        and ttmb.short_period = 8 and ttmb.long_period=89 and ttmb.signal_period=89 \
    LEFT JOIN \
    `idc.macd` as ttmc on ttmc.symbol = stock.symbol and ttmc.effective_date = stock.effective_date \
        and ttmc.short_period = 8 and ttmc.long_period=144 and ttmc.signal_period=144 \
    ".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [ ]:
def drawInterChart(df):
    trace = go.Ohlc(x=df.date, open=df.open, high=df.high, low=df.low, close=df.close)
    data = [trace]
    layout = {
        'title': 'Stock Chart',
        'yaxis': {'title': 'Price'},
        'shapes': [{
            'x0': '2016-12-09', 'x1': '2016-12-09',
            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
            'line': {'color': 'rgb(30,30,30)', 'width': 1}
        }],
        'annotations': [{
            'x': '2016-12-09', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
            'showarrow': False, 'xanchor': 'left',
            'text': 'Increase Period Begins'
        }]
    }
    fig = dict(data=data, layout=layout)
    
    # 
    # embbeded chart
    #plotly.offline.iplot(fig)
    
    # standalone chart
    plotly.offline.plot(fig)
    
    # online chart 
    # py.iplot(fig, filename='appl-recession-candlestick')

In [41]:
df = load_stockdata('MSFT')
df['macd_up'] = df.macd > df.macd.shift(-1)
df['squeeze'] = (df.bb_lower >= df.kc_2_lower) & (df.bb_upper <= df.kc_2_upper)
df['squeeze_value'] = 0
df.head(20)


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,...,kc_3_lower,kc_1_upper,kc_1_lower,macd,wave_a,wave_b,wave_c,macd_up,squeeze,squeeze_value
effective_date,,,,,,,,,,,,,,,,,,,,,
2018-02-23,MSFT,92.750,94.0700,92.3600,94.06,25779955.0,91.4595,96.304368,86.614632,91.155589,...,84.161341,93.487005,88.824173,NaN,NaN,NaN,NaN,False,False,0
2018-02-22,MSFT,92.050,92.7300,91.3600,91.74,22723756.0,91.3730,96.089126,86.656874,90.849861,...,83.855286,93.181386,88.518336,NaN,NaN,NaN,NaN,False,False,0
2018-02-21,MSFT,92.980,93.3595,91.4900,91.49,26081953.0,91.3770,96.094500,86.659500,90.756163,...,83.539699,93.161651,88.350675,NaN,NaN,NaN,NaN,False,False,0
2018-02-20,MSFT,91.475,93.0600,91.0100,92.72,30107057.0,91.3975,96.120346,86.674654,90.678917,...,83.338763,93.125635,88.232199,NaN,NaN,NaN,NaN,False,False,0
2018-02-16,MSFT,92.450,93.5000,91.8000,92.00,30188535.0,91.3420,96.027316,86.656684,90.464066,...,83.032361,92.941301,87.986831,NaN,NaN,NaN,NaN,False,False,0
2018-02-15,MSFT,91.210,92.7200,90.6200,92.66,27407914.0,91.2420,95.952178,86.531822,90.302389,...,82.691323,92.839411,87.765367,NaN,NaN,NaN,NaN,False,False,0
2018-02-14,MSFT,88.510,90.9900,88.4100,90.81,34314101.0,91.1140,95.802170,86.425830,90.054219,...,82.342302,92.624858,87.483580,NaN,NaN,NaN,NaN,False,False,0
2018-02-13,MSFT,88.930,90.0000,87.8000,89.83,26200053.0,91.0805,95.786422,86.374578,89.974663,...,82.264906,92.544582,87.404744,NaN,NaN,NaN,NaN,False,False,0
2018-02-12,MSFT,88.735,89.7800,87.9295,89.13,35501368.0,91.0065,95.833732,86.179268,89.989891,...,82.194769,92.588265,87.391517,NaN,NaN,NaN,NaN,False,False,0


In [42]:
#[macd, up in df.macd_histogram, df.macd_up]
df.to_csv("GPC.csv")

In [ ]:
#[ row.macd > 0 & row.macd_up for row in df]
#df['color'] = ('green' if (df['macd'] > 0).bool() else 'red')
#[row['macd'] for row in df.iterrows()]
colordict = []
for index, row in df.iterrows():
    
    color =  '#029107' if (row['macd']>=0 and row['macd_up']==True) else \
            ('#0599b7' if (row['macd']>=0 and row['macd_up']==False) else \
            ('#871001' if (row['macd']<0 and row['macd_up']==False) else '#d35004' ))
    colordict.append(color)
    #print(index, row['macd'], row['macd_up'], color)
    

In [6]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [39]:
def multi_chart(df):
    
    style_bollingerbands_middle = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'dot')
    style_bollingerbands_upper = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'line')
    style_bollingerbands_lower = dict( color = ('rgb(91, 154, 255)'), width = 1, dash = 'line')

    style_kc_middle = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'dot')
    style_kc2_upper = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'line')
    style_kc2_lower = dict( color = ('rgb(244, 118, 73)'), width = 1, dash = 'line')

    style_kc3_upper = dict( color = ('rgb(46, 155, 53)'), width = 1, dash = 'line')
    style_kc3_lower = dict( color = ('rgb(68, 206, 77)'), width = 1, dash = 'line')

    style_kc1_upper = dict( color = ('rgb(244, 78, 66)'), width = 1, dash = 'line')
    style_kc1_lower = dict( color = ('rgb(198, 70, 61)'), width = 1, dash = 'line')

    trace_candle = go.Candlestick(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#51150e'))
                         ,hoverinfo='y'
                        )

    trace_ohlc = go.Ohlc(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#51150e'))
                         #,hoverinfo='none'
                        )
    
    trace_bb_middle = go.Scatter(x=df.index, y=df.bb_middle, name='BB Middle', legendgroup='Bollinger Bands',
                                 showlegend=False, opacity=0.5, hoverinfo='none',
                                 line = style_bollingerbands_middle
                                )
    trace_bb_upper = go.Scatter(x=df.index, y=df.bb_upper, name='BB Upper', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_upper
                               )
    trace_bb_lower = go.Scatter(x=df.index, y=df.bb_lower, name='BB Lower', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_lower
                               )
    
    trace_kc_middle = go.Scatter(x=df.index, y=df.kc_middle, name='KC Middle', legendgroup='Keltner Channels 2ATR',
                                 showlegend=False, opacity=0.5, hoverinfo='none',
                                 line = style_kc_middle
                                )
    trace_kc_2atr_upper = go.Scatter(x=df.index, y=df.kc_2_upper, name='KC 2ATR Upper', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_upper
                                    )
    trace_kc_2atr_lower = go.Scatter(x=df.index, y=df.kc_2_lower, name='KC 2ATR Lower', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_lower
                                    )

    # create volume bar chart on the bottom
    tarce_volume = go.Bar(x=df.index, y=df.adj_volume, name='volume', showlegend=False, marker=dict(color='rgb(87, 91, 130)'),
                          yaxis = 'y2'
                         )

    # color dict for MACD and Squeeze indicator
    colordict = []
    colorsqueeze = []
    for index, row in df.iterrows():

        color =  '#029107' if (row['macd']>=0 and row['macd_up']==True) else \
                ('#0599b7' if (row['macd']>=0 and row['macd_up']==False) else \
                ('#600601' if (row['macd']<0 and row['macd_up']==False) else '#d35004' ))
        colordict.append(color)

        color_s = '#f21104' if row['squeeze'] else '#404241'
        colorsqueeze.append(color_s)
        #df['squeeze_color'] = '#930c02' if df.squeeze else '#0cb703'
        
    trace_macd = go.Bar(x=df.index, y=df.macd, name='MACD(12/26/9)', showlegend=False, 
                        marker=dict(color=colordict),
                        yaxis = 'y3'
                       )

    trace_squeeze = go.Scatter(x=df.index, y=df.squeeze_value, name='', showlegend=False, 
                        mode = 'markers', marker=dict(size=6, color=colorsqueeze), hoverinfo='none',
                        yaxis = 'y3'
                       )

    trace_wave_a = go.Bar(x=df.index, y=df.wave_a, name='WAVE A(8/34/34)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_a]),
                    yaxis = 'y4'
                   )

    trace_wave_b = go.Bar(x=df.index, y=df.wave_b, name='WAVE B(8/89/89)', showlegend=False, 
                marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_b]),
                yaxis = 'y5'
               )
    trace_wave_c = go.Bar(x=df.index, y=df.wave_c, name='WAVE C(8/144/144)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_c]),
                    yaxis = 'y6'
                   )

    
    data = [trace_candle, trace_bb_middle, trace_kc_middle,
            trace_bb_upper, trace_bb_lower, trace_kc_2atr_upper, trace_kc_2atr_lower, 
            #tarce_volume, 
            trace_macd, trace_squeeze, trace_wave_a, trace_wave_b, trace_wave_c]
    
    
    axis_template = dict(showgrid=True, zeroline=True, showline=True, rangeslider=dict(visible=False), fixedrange=True)
    layout = go.Layout(
        title = 'John Carter TTM Squeeze',
        height=1000,
        paper_bgcolor='#000', plot_bgcolor='#000',
        margin = go.Margin(l=80,r=30,t=50,b=100),
        xaxis = axis_template,
        yaxis = dict(domain=[0.45,1]),
        # yaxis2 = dict(domain=[0.4,0.5], visible=False),
        yaxis3 = dict(domain=[0.3,0.4], title='MACD'),
        yaxis4 = dict(domain=[0.2,0.3], visible=True, title="WAVEA"),
        yaxis5 = dict(domain=[0.1,0.2], visible=True, title="WAVEB"),
        yaxis6 = dict(domain=[0,0.1], visible=True, title="WAVEC"),
        legend = dict(x=0.05, y=1.0)
    )

    fig = go.Figure(data=data, layout=layout)
    # standalone chart
    plotly.offline.plot(fig, "multi chart")


In [43]:
multi_chart(df)